In [1]:
import os
from datetime import datetime
import time
import tqdm
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder
import numpy as np
import torch

In [2]:
class CFG:
    path = "../../input/data/train_dataset"
    train = "train_data.csv"
    test = "test_data.csv"

In [3]:
csv_file_path = os.path.join(CFG.path, CFG.train)
df = pd.read_csv(csv_file_path)
df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225


### newID 추가

In [4]:
seq_len = 20

In [5]:
%%time
new_id = 0
before = 0
count = 0
new_user = []

# df를 list로 뽑아와서 처리하는게 훨씬 빠르다!
userid = df.userID.tolist()
# userid.reverse()
print(userid[:10])

for u in userid:
    if (count == seq_len) or (u != before):
        new_id += 1
        count = 0
        
    new_user.append(new_id)
    count += 1
    
    before = u
new_user[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
CPU times: user 660 ms, sys: 32 ms, total: 692 ms
Wall time: 696 ms


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [6]:
# %%time
# new_id = 0
# before = 0
# count = 0
# new_user = []

# for u in reversed(df.userID):
#     if (count == seq_len) or (u != before):
#         new_id += 1
#         count = 0
        
#     new_user.append(new_id)
#     count += 1
    
#     before = u
# new_user[:10]

In [7]:
# new_user.reverse()
# max_user = max(new_user)
# new_user = [max_user - n for n in new_user]

In [8]:
# df['newID'] = new_user

In [9]:
# new_df = df.copy()
df['newID'] = new_user
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,newID
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,0
...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,116486
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,116486
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,116486
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,116486


In [10]:
df.to_csv('temp_2.csv', index=False)

### assessmentitemID 분리
- A 제외하고 앞의 6자리 시험지번호
- 6자리 시험지번호 중 앞의 3자리는 0x0 형태로, x는 0~9의 대분류를 나타낸다
- 그 뒤의 마지막 3자리 문항 번호

In [11]:
df.assessmentItemID

0          A060001001
1          A060001002
2          A060001003
3          A060001004
4          A060001005
              ...    
2266581    A030071005
2266582    A040165001
2266583    A040165002
2266584    A040165003
2266585    A040165004
Name: assessmentItemID, Length: 2266586, dtype: object

In [12]:
# 시험지 번호
df['papernum'] = df.assessmentItemID.apply(lambda x: x[1:7])
df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,newID,papernum
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0,060001
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,0,060001
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,0,060001
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,0,060001
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,0,060001


In [13]:
# 대분류 - head
df['head'] = df.assessmentItemID.apply(lambda x: x[1:4])
df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,newID,papernum,head
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0,060001,060
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,0,060001,060
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,0,060001,060
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,0,060001,060
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,0,060001,060


In [14]:
# mid
df['mid'] = df.assessmentItemID.apply(lambda x: x[4:7])
df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,newID,papernum,head,mid
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0,060001,060,001
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,0,060001,060,001
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,0,060001,060,001
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,0,060001,060,001
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,0,060001,060,001


In [15]:
# tail - 시험지 번호
df['tail'] = df.assessmentItemID.apply(lambda x: x[7:])
df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,newID,papernum,head,mid,tail
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0,060001,060,001,001
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,0,060001,060,001,002
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,0,060001,060,001,003
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,0,060001,060,001,004
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,0,060001,060,001,005


In [32]:
# head_tail
df['head_tail'] = df.assessmentItemID.apply(lambda x: x[:4]+x[7:])
df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,newID,papernum,head,mid,tail,tail_prob,test_split_id,UserTestCumtestnum,TestSize,Retest,boundary,time_diff,head_tail
0,0,A060001001,A060000001,1,1585009031,7224,0,060001,060,001,001,0.749916,0,0,6,0,0,3.0,A060001
1,0,A060001002,A060000001,1,1585009034,7225,0,060001,060,001,002,0.720062,0,1,6,0,1,3.0,A060002
2,0,A060001003,A060000001,1,1585009042,7225,0,060001,060,001,003,0.687773,0,2,6,0,2,8.0,A060003
3,0,A060001004,A060000001,1,1585009049,7225,0,060001,060,001,004,0.663364,0,3,6,0,3,7.0,A060004
4,0,A060001005,A060000001,1,1585009056,7225,0,060001,060,001,005,0.599134,0,4,6,0,4,7.0,A060005


In [33]:
# mid_tail
df['mid_tail'] = df.assessmentItemID.apply(lambda x: x[4:])
df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,newID,papernum,head,mid,tail,tail_prob,test_split_id,UserTestCumtestnum,TestSize,Retest,boundary,time_diff,head_tail,mid_tail
0,0,A060001001,A060000001,1,1585009031,7224,0,060001,060,001,001,0.749916,0,0,6,0,0,3.0,A060001,001001
1,0,A060001002,A060000001,1,1585009034,7225,0,060001,060,001,002,0.720062,0,1,6,0,1,3.0,A060002,001002
2,0,A060001003,A060000001,1,1585009042,7225,0,060001,060,001,003,0.687773,0,2,6,0,2,8.0,A060003,001003
3,0,A060001004,A060000001,1,1585009049,7225,0,060001,060,001,004,0.663364,0,3,6,0,3,7.0,A060004,001004
4,0,A060001005,A060000001,1,1585009056,7225,0,060001,060,001,005,0.599134,0,4,6,0,4,7.0,A060005,001005


### Tail의 정답률

In [16]:
def percentile(s):
        return np.sum(s) / len(s)

In [17]:
prob_groupby = df.groupby('tail').agg({
    'answerCode': percentile,
})

In [18]:
prob_groupby

,answerCode
tail,
001,0.749916
002,0.720062
003,0.687773
004,0.663364
005,0.599134
006,0.555685
007,0.515399
008,0.457156
009,0.481729


In [19]:
tail_prob_list = prob_groupby['answerCode'].unique().tolist()
tail_prob_list

[0.7499159392610725,
 0.7200624305877532,
 0.6877731137660101,
 0.6633640334861579,
 0.5991342375494652,
 0.5556853346277532,
 0.5153987146104893,
 0.45715562174236785,
 0.4817288374871043,
 0.5278921752152752,
 0.4806085918854415,
 0.37037037037037035,
 0.20074349442379183]

In [20]:
df['tail_prob'] = df['tail'].apply(lambda x: tail_prob_list[int(x)-1])

In [21]:
df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,newID,papernum,head,mid,tail,tail_prob
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0,060001,060,001,001,0.749916
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,0,060001,060,001,002,0.720062
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,0,060001,060,001,003,0.687773
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,0,060001,060,001,004,0.663364
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,0,060001,060,001,005,0.599134


In [22]:
df.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'newID', 'papernum', 'head', 'mid', 'tail',
       'tail_prob'],
      dtype='object')

### userID에서 testID n개 기준으로 묶기
- test에서 testID 마지막을 기준으로 예측하므로, 이렇게 묶어서 마지막 testID를 예측하게 하면 성능이 오를 것이라는 예측

In [23]:
%%time
userid = df.userID.tolist()
testid = df.testId.tolist()
testid_set = set()

new_id = 0
before = 0

new_user = []

for i in range(len(userid)):
    testid_set.add(testid[i])
    if (len(testid_set) > 3) or (userid[i] != before):
        new_id += 1
        testid_set = set()
    
    new_user.append(new_id)
    before = userid[i]
    
len(new_user)

CPU times: user 1.11 s, sys: 20 ms, total: 1.13 s
Wall time: 1.13 s


2266586

In [24]:
%%time
userid = df.userID
testid = df.testId
testid_set = set()

new_id = 0
before = 0

new_user = []

for i in range(len(userid)):
    testid_set.add(testid[i])
    if (len(testid_set) > 3) or (userid[i] != before):
        new_id += 1
        testid_set = set()
    
    new_user.append(new_id)
    before = userid[i]
    
len(new_user)

CPU times: user 25.5 s, sys: 0 ns, total: 25.5 s
Wall time: 25.5 s


2266586

In [25]:
df['test_split_id'] = new_user
df.to_csv('temp_3.csv', index=False)

### time_diff

In [26]:
def convert_time(s):
    timestamp = time.mktime(datetime.strptime(s, '%Y-%m-%d %H:%M:%S').timetuple())
    return int(timestamp)

df['Timestamp'] = df['Timestamp'].apply(convert_time)

In [27]:
time_diff = df.groupby(['userID', 'head', 'mid'])['Timestamp'].diff()
time_diff

0           NaN
1           3.0
2           8.0
3           7.0
4           7.0
           ... 
2266581    24.0
2266582     NaN
2266583    11.0
2266584    46.0
2266585    73.0
Name: Timestamp, Length: 2266586, dtype: float64

In [29]:
## find boundary
# userID, testId 별 푼 문항의 누적 합
df['UserTestCumtestnum'] = df.groupby(['userID','testId'])['answerCode'].cumcount()
testId2maxlen = df[['assessmentItemID', 'testId']].drop_duplicates().groupby('testId').size()

# test의 문항 수
df['TestSize'] = df.testId.map(testId2maxlen)

# user가 같은 test를 여러 번 푼 것인지 나타낸 변수 (처음 품 : 0, 두번 품 : 1, 세번 품 : 2)
df['Retest'] = df['UserTestCumtestnum'] // df['TestSize']

# boundary
df['boundary'] = [u % t if t != 0 else 0.0 for t, u in zip(df['TestSize'], df['UserTestCumtestnum'])]

In [30]:
df['time_diff'] = time_diff

df['time_diff'].fillna(method='bfill', inplace=True)
# df['time_diff'].fillna(0, inplace=True) -> 성능하락

df['time_diff'] =df['time_diff'].map(lambda x: 600 if x>600 else x)

In [31]:
df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,newID,papernum,head,mid,tail,tail_prob,test_split_id,UserTestCumtestnum,TestSize,Retest,boundary,time_diff
0,0,A060001001,A060000001,1,1585009031,7224,0,060001,060,001,001,0.749916,0,0,6,0,0,3.0
1,0,A060001002,A060000001,1,1585009034,7225,0,060001,060,001,002,0.720062,0,1,6,0,1,3.0
2,0,A060001003,A060000001,1,1585009042,7225,0,060001,060,001,003,0.687773,0,2,6,0,2,8.0
3,0,A060001004,A060000001,1,1585009049,7225,0,060001,060,001,004,0.663364,0,3,6,0,3,7.0
4,0,A060001005,A060000001,1,1585009056,7225,0,060001,060,001,005,0.599134,0,4,6,0,4,7.0


### userID를 testId로 나눈 test_split_id의 정답률

In [46]:
df.groupby('test_split_id')

In [47]:
prob_groupby = df.groupby('test_split_id').agg({
    'answerCode': percentile,
})

In [52]:
prob_groupby

,answerCode
test_split_id,
0,0.900000
1,0.900000
2,0.695652
3,0.772727
4,0.521739
...,...
126403,0.466667
126404,0.375000
126405,0.750000


In [48]:
test_split_prob_list = prob_groupby['answerCode'].tolist()
test_split_prob_list[:10]

[0.9,
 0.9,
 0.6956521739130435,
 0.7727272727272727,
 0.5217391304347826,
 0.5652173913043478,
 0.6521739130434783,
 0.47619047619047616,
 0.6842105263157895,
 0.5909090909090909]

In [53]:
df['test_split_prob'] = df['test_split_id'].apply(lambda x: test_split_prob_list[int(x)])

In [54]:
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,newID,papernum,head,mid,...,tail_prob,test_split_id,UserTestCumtestnum,TestSize,Retest,boundary,time_diff,head_tail,mid_tail,test_split_prob
0,0,A060001001,A060000001,1,1585009031,7224,0,060001,060,001,...,0.749916,0,0,6,0,0,3.0,A060001,001001,0.900000
1,0,A060001002,A060000001,1,1585009034,7225,0,060001,060,001,...,0.720062,0,1,6,0,1,3.0,A060002,001002,0.900000
2,0,A060001003,A060000001,1,1585009042,7225,0,060001,060,001,...,0.687773,0,2,6,0,2,8.0,A060003,001003,0.900000
3,0,A060001004,A060000001,1,1585009049,7225,0,060001,060,001,...,0.663364,0,3,6,0,3,7.0,A060004,001004,0.900000
4,0,A060001005,A060000001,1,1585009056,7225,0,060001,060,001,...,0.599134,0,4,6,0,4,7.0,A060005,001005,0.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,1591339821,438,116486,030071,030,071,...,0.599134,126407,4,5,0,4,24.0,A030005,071005,0.555556
2266582,7441,A040165001,A040000165,1,1597971999,8836,116486,040165,040,165,...,0.749916,126407,0,4,0,0,11.0,A040001,165001,0.555556
2266583,7441,A040165002,A040000165,1,1597972010,8836,116486,040165,040,165,...,0.720062,126407,1,4,0,1,11.0,A040002,165002,0.555556
2266584,7441,A040165003,A040000165,1,1597972056,8836,116486,040165,040,165,...,0.687773,126407,2,4,0,2,46.0,A040003,165003,0.555556


In [55]:
df.to_csv('temp_4.csv', index=False)